<a href="https://colab.research.google.com/github/shinobu9/mipt-ml-course/blob/main/homeworks/hw04_umt/Lab_NLP_part1_Embedding_based_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/), [Valery Marchenkov](https://www.linkedin.com/in/vmarchenkoff/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages.

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for English and French languages.

In [1]:
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gzip -d cc.en.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
!gzip -d cc.fr.300.vec.gz

--2023-04-14 15:23:10--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.10, 13.227.219.59, 13.227.219.70, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1325960915 (1.2G) [binary/octet-stream]
Saving to: ‘cc.en.300.vec.gz’

cc.en.300.vec.gz    100%[===================>]   1.23G   190MB/s    in 6.5s    

2023-04-14 15:23:17 (193 MB/s) - ‘cc.en.300.vec.gz’ saved [1325960915/1325960915]

--2023-04-14 15:23:56--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.227.219.70, 13.227.219.59, 13.227.219.10, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.227.219.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1287757366 (1.2G) [binary/octet-stream]
Saving to: ‘c

After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

In [2]:
from gensim.models import KeyedVectors
import numpy as np


en_emb = KeyedVectors.load_word2vec_format("cc.en.300.vec")
fr_emb = KeyedVectors.load_word2vec_format("cc.fr.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

In [3]:
august_embedding = en_emb["august"]
august_embedding.shape, august_embedding[:5]

((300,), array([-0.0522,  0.0364, -0.1252,  0.0053,  0.0382], dtype=float32))

In [4]:
en_emb.most_similar([august_embedding])

[('august', 0.9999999403953552),
 ('september', 0.8252838850021362),
 ('october', 0.8111193180084229),
 ('june', 0.8050147891044617),
 ('july', 0.797055184841156),
 ('november', 0.788363516330719),
 ('february', 0.7831973433494568),
 ('december', 0.7824540138244629),
 ('january', 0.7743154168128967),
 ('april', 0.7621643543243408)]

The latter function also allows you to vary the amount of closest words via the `topn` argument:

In [5]:
en_emb.most_similar([august_embedding], topn=3)

[('august', 0.9999999403953552),
 ('september', 0.8252838850021362),
 ('october', 0.8111193180084229)]

Another feature of `KeyedVectors` is that it allows to compute embeddings for multiple words simultaneously:

In [6]:
en_emb[["august", "september"]].shape

(2, 300)

Everything above is true for the embeddings for French language.

In [7]:
fr_emb.most_similar([fr_emb["aout"]])

[('aout', 1.0),
 ('Aout', 0.8249964118003845),
 ('juillet', 0.8109882473945618),
 ('fevrier', 0.8072442412376404),
 ('septembre', 0.7838520407676697),
 ('août', 0.779176652431488),
 ('juin', 0.7692081332206726),
 ('octobre', 0.7597455382347107),
 ('decembre', 0.7595790028572083),
 ('avril', 0.7390779256820679)]

However, french and english embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in French and English:

In [8]:
fr_emb.most_similar([en_emb["august"]])

[('2003Pays', 0.23082853853702545),
 ('Montsoriu', 0.22505579888820648),
 ('2015Pays', 0.22218400239944458),
 ('2013Genre', 0.2095685601234436),
 ('AdiCloud', 0.2018650770187378),
 ('Bagua', 0.20061466097831726),
 ('2003Paysans', 0.2001495361328125),
 ('ValenceLa', 0.2001476287841797),
 ('Luddites', 0.19998176395893097),
 ('Guadalquivir', 0.19875513017177582)]

## Translation

We'll build a simple translator, which will try to predict the french embedding from the english one. For this we'll need a dataset of word pairs.

In [9]:
def load_word_pairs(filename):
    en_fr_pairs = []
    en_vectors = []
    fr_vectors = []
    with open(filename, "r") as inpf:
        for line in inpf:
            en, fr = line.rstrip().split(" ")
            if en not in en_emb or fr not in fr_emb:
                continue
            en_fr_pairs.append((en, fr))
            en_vectors.append(en_emb[en])
            fr_vectors.append(fr_emb[fr])
    return en_fr_pairs, np.array(en_vectors), np.array(fr_vectors)

We will train our model to predict embedding for the french word from embedding of its english counterpart. For this reason we split our train and test data into english and french words and compute corresponding embeddings to obtain `X` (english embeddings) and `y` (french embeddings).

In [10]:
!wget -O en-fr.train.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
!wget -O en-fr.test.txt https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt

--2023-04-14 15:39:30--  https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178608 (174K) [text/plain]
Saving to: ‘en-fr.train.txt’

en-fr.train.txt     100%[===================>] 174.42K  --.-KB/s    in 0.009s  

2023-04-14 15:39:30 (19.4 MB/s) - ‘en-fr.train.txt’ saved [178608/178608]

--2023-04-14 15:39:30--  https://raw.githubusercontent.com/girafe-ai/ml-course/23s_nes/homeworks/hw04_umt/en-fr.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response...

In [11]:
en_fr_train, X_train, Y_train = load_word_pairs("en-fr.train.txt")
en_fr_test, X_test, Y_test = load_word_pairs("en-fr.test.txt")

In [12]:
en_fr_train[33:44]

[('which', 'lesquels'),
 ('which', 'laquelle'),
 ('which', 'lequel'),
 ('also', 'également'),
 ('also', 'aussi'),
 ('also', 'egalement'),
 ('were', 'étaient'),
 ('but', 'mais'),
 ('have', 'avoir'),
 ('have', 'ont'),
 ('one', 'un')]

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called [Procrustes problem](https://en.wikipedia.org/wiki/Orthogonal_Procrustes_problem):

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

In [18]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
mapping = LinearRegression(fit_intercept=False)
mapping.fit(X_train, Y_train)

LinearRegression(fit_intercept=False)

Let's take a look at neigbours of the vector of word _"august"_ (_"aout"_ in French) after linear transform.

In [34]:
august = mapping.predict(en_emb["august"].reshape(1, -1))
fr_emb.most_similar(august)

[('juin', 0.7553410530090332),
 ('aout', 0.7527693510055542),
 ('juillet', 0.7500796318054199),
 ('septembre', 0.7482382655143738),
 ('mars', 0.74159836769104),
 ('octobre', 0.7395485043525696),
 ('novembre', 0.7313361763954163),
 ('février', 0.7296543717384338),
 ('janvier', 0.7272254824638367),
 ('avril', 0.7249919176101685)]

In [35]:
fr = fr_emb.most_similar(august)

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed english embedding we count how many right target pairs are found in top N nearest neighbours in french embedding space).

In [41]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(en_word_0, fr_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    total = len(pairs)
    correct = 0
    for i in range(total):
        pair = pairs[i]
        predicted_vector = mapped_vectors[i]
        
        # YOUR CODE HERE
        ms_pred = fr_emb.most_similar(predicted_vector, topn=topn)
        top = [j[0] for j in ms_pred]

        if pair[1] in top: correct+=1
        

    return correct / total

In [42]:
assert precision([("august", "aout")], august, topn=5) == 1.0
assert precision([("august", "aout")], august, topn=9) == 1.0
assert precision([("august", "aout")], august, topn=10) == 1.0

Note that our `precision` function accepts lists of pairs of words, whereas we have dataframes. However, it is not a problem: we can get a list (actually, numpy array) of pairs via the `values` property.

In [43]:
assert precision(en_fr_test[:100], X_test[:100]) == 0.0
assert precision(en_fr_test[:100], Y_test[:100]) == 1.0

Let's see how well our model is doing.

In [44]:
precision_top1 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 1)
precision_top5 = precision(en_fr_test[:100], mapping.predict(X_test[:100]), 5)

In [45]:
print(precision_top1)
print(precision_top5)

0.38
0.67


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [49]:
import numpy as np


# YOUR CODE HERE
# Compute the orthogonal mapping (W^T)^* as defined in formula above.
u, s, vt = np.linalg.svd(X_train.T@Y_train, full_matrices=True)
mapping_svd = u@vt

Now our `mapping` is just a numpy array, meaning that it has no `predict` method. However, from the formulae above we know, that prediction is done using the matrix multiplication:

In [56]:
fr_emb.most_similar([np.matmul(en_emb['august'], mapping_svd)],topn=1)[0][0]

'aout'

Now let's compute our precision values and see, whether our trick did improve the results.

In [51]:
print(precision(en_fr_test[:100], np.matmul(X_test[:100], mapping_svd)))
print(precision(en_fr_test[:100], np.matmul(X_test[:100], mapping_svd), 5))

0.36
0.68


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Now let's translate these sentences word-by-word. Before that, however, don't forget to tokenize your sentences. For that you may (or may not) find the `nltk.tokenize.WordPunctTokenizer` to be very useful.

In [132]:
def translate(sentence):
    """
    :args:
        sentence - sentence in English (str)
    :returns:
        translation - sentence in French (str)

    * find english embedding for each word in sentence
    * transform english embedding vector
    * find nearest french word and replace
    """
    translated = []

    # YOUR CODE HERE
    for word in sentence.split(' '):
      if word in en_emb.key_to_index.keys():
        translated.append(fr_emb.most_similar([np.matmul(en_emb[word], mapping_svd)],topn=1)[0][0])
      else:
        translated.append(word)


    return " ".join(translated)

In [133]:
assert translate(".") == "."
assert translate("I walk around Paris") == "je marcher autour Paris"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [134]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords, twitter_samples
import re
import string

nltk.download('twitter_samples')
nltk.download('stopwords')

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    wnl = WordNetLemmatizer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
        # if word not in string.punctuation:
            # tweets_clean.append(word)
            # stem_word = stemmer.stem(word)  # stemming word
            # tweets_clean.append(stem_word)
            lemmatize_word = wnl.lemmatize(word, pos="v")
            tweets_clean.append(lemmatize_word)

    return " ".join(tweets_clean)

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [135]:
twitter_samples.strings('positive_tweets.json')[10:15]

['#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)',
 "Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY",
 '@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)',
 "@jjulieredburn Perfect, so you already know what's waiting for you :)",
 'Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0']

In [136]:
!python3 -m nltk.downloader wordnet
!unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

/usr/lib/python3.9/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /root/nltk_data/corpora/wordnet.zip
replace /root/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [137]:
for i in twitter_samples.strings('positive_tweets.json')[10:15]:
    print(i, process_tweet(i), sep='\n\n', end='\n-----------------\n')

#FollowFriday @wncer1 @Defense_gouv for being top influencers in my community this week :)

followfriday top influencers community week :)
-----------------
Who Wouldn't Love These Big....Juicy....Selfies :) - http://t.co/QVzjgd1uFo http://t.co/oWBL11eQRY

love big ... juicy ... selfies :)
-----------------
@Mish23615351  follow @jnlazts &amp; http://t.co/RCvcYYO0Iq follow u back :)

follow
-----------------
@jjulieredburn Perfect, so you already know what's waiting for you :)

perfect already know what's wait :)
-----------------
Great new opportunity for junior triathletes aged 12 and 13 at the Gatorade series! Get your entries in :) http://t.co/of3DyOzML0

great new opportunity junior triathletes age 12 13 gatorade series get entries :)
-----------------


Your translation:

In [138]:
for i in twitter_samples.strings('positive_tweets.json')[:10]:
    print(i, process_tweet(i), translate(process_tweet(i)), sep='\n\n', end='\n-----------------\n')

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

followfriday top engage members community week :)

twitt top engager membres communauté semaine okk
-----------------
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!

hey jam odd :/ please call contact centre 02392441234 able assist :) many thank

hey jam bizarre :/ veuillez appeler contacter centre 02392441234 puisse aider okk nombreux remercie
-----------------
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!

listen last night :) bleed amaze track scotland

écouter dernière nuit okk saigner époustoufler track ecosse
-----------------
@97sides CONGRATS :)

congrats :)

félicitation okk
-----------------
yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days

yeaaah yipppy accnt verify r

Great! 